In [5]:

#https://turbomaze.github.io/word2vecjason 

#Gustav Soderstorm : Spotify 의 LLM 작동 방식에 대한 설명 추천 
#Explained : the conspiracy to make AI seem harder than it is By Gustave Soderstorm 
#from langchain.chat_models import ChatOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import  UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings,CacheBackedEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

# splitter = RecursiveCharacterTextSplitter(
#     chunk_size = 200,
#     chunk_overlap = 50,
# )

llm = ChatOpenAI(temperature=0.1)
cache_dir = LocalFileStore("./.cache/")


# https://platform.openai.com/tokenizer
splitter = CharacterTextSplitter.from_tiktoken_encoder(  #AI model 이 이해하는 토큰 방식을 사용해서 
    separator="\n\n",
    chunk_size = 600,
    chunk_overlap = 50,

)


loader = UnstructuredFileLoader("./files/it_infra.pptx")
docs = loader.load_and_split(text_splitter=splitter)
embedder = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embedder,cache_dir,
)

#vectorstores = Chroma.from_documents(docs,cached_embeddings) 
#Chroma 보다 FAISS 가 성능이 좋은 듯 . cache 파일 지우고 다시 시도 
vectorstores = FAISS.from_documents(docs,cached_embeddings) 


#Langsmith 에 가입하여 API Key 생성후 env 저장 
#RetrievalQA 에서 chain type (Stuff, Refine, Map Reduce , Map rank ) 을 바꿔가며 사용가능 

retriever = vectorstores.as_retriever()
prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant. Answer question using only the following context.If you know know the answer just say you don't know. Don't make it up :\n\n{context} "),
    ("human","{question}")
]
)

chain = {"context": retriever,"question": RunnablePassthrough()} | prompt | llm 

# chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     #chain_type="refine",
#     #
#     # chain_type="map_reduce",
#     retriever=vectorstores.as_retriever(),
# )

chain.invoke("What is the purpose of VIS meeting?")
#해당 실행 결과 조회 사이트 
# https://smith.langchain.com/o/c5060851-b86a-4545-a372-4071ad43f851/projects/p/7fc9897f-187b-4dc1-9d40-14a5caec06de?timeModel=%7B%22duration%22%3A%227d%22%7D

AIMessage(content='The purpose of the VIS meeting is to systematically verify the reliability, security, compliance, financial stability, and performance of vendors before extending the maintenance contract.')

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import  UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings,CacheBackedEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough


llm = ChatOpenAI(temperature=0.1)
cache_dir = LocalFileStore("./.cache/")


splitter = CharacterTextSplitter.from_tiktoken_encoder(  
    separator="\n\n",
    chunk_size = 600,
    chunk_overlap = 50,

)


loader = UnstructuredFileLoader("./files/document.txt")
docs = loader.load_and_split(text_splitter=splitter)

embedder = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embedder,cache_dir,
)


vectorstores = FAISS.from_documents(docs,cached_embeddings) 



retriever = vectorstores.as_retriever()
prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant. Answer question using only the following context.If you know know the answer just say you don't know. Don't make it up :\n\n{context} "),
    ("human","{question}")
]
)

chain.invoke("What is the purpose of VIS meeting?")

AIMessage(content='The purpose of the VIS meeting is to systematically verify the reliability, security, compliance, financial stability, and performance of vendors before making decisions on extending IT infrastructure maintenance.')